### Multi-Year Lake Ice Phenology Data Export
## Part 2: GEE and xarray Processing and Export (2019-2023)

**Goal:** Export S1 + S2 + ERA5 data for North Slope lakes across 5 years

**Strategy:**
- Process chunks independently (spatial parallelization)
- Export one year at a time per chunk
- Use efficient spatial filtering (only process S2 images that overlap lakes)
- Export ERA5 via xarray (more efficient)
- Total GEE exports: ~21 chunks × 5 years × 2 datasets (S1, S2) = ~210 exports


**Data sources:**
- ERA5-Land (temperature)
- Sentinel-1 GRD (SAR)
- Sentinel-2 SR Harmonized (optical, for NDSI)

**Years:** 2019, 2020, 2021, 2022, 2023

---
## Setup

In [60]:
import ee
import pandas as pd
import numpy as np
import geopandas as gpd
from datetime import datetime
import time
import xarray as xr
import subprocess
import sys
import warnings

# Initialize Earth Engine
ee.Initialize()

print("Imports successful!")
print(f"Earth Engine initialized: {ee.String('GEE Initialized').getInfo()}")

httplib2 transport does not support per-request timeout. Set the timeout when constructing the httplib2.Http instance.
httplib2 transport does not support per-request timeout. Set the timeout when constructing the httplib2.Http instance.
httplib2 transport does not support per-request timeout. Set the timeout when constructing the httplib2.Http instance.
httplib2 transport does not support per-request timeout. Set the timeout when constructing the httplib2.Http instance.


Imports successful!
Earth Engine initialized: GEE Initialized


In [61]:
# Configuration
BUCKET = 'wustl-eeps-geospatial'
BASE_PATH = 'thermokarst_lakes'
CHUNKS_PATH = f'gs://{BUCKET}/{BASE_PATH}/processed/chunks'
OUTPUT_PATH = f'{BASE_PATH}/exports'
GEE_ASSET_PATH = 'projects/eeps-geospatial/assets/lake_geometries'
YEARS = [2019, 2020, 2021, 2022, 2023]
n_chunks = 21

# S2 thresholds
S2_CLOUD_THRESHOLD = 30
S2_CLOUD_PROB_THRESHOLD = 40
S2_NDSI_THRESHOLD = 0.4

# Buffer distances (meters)
INTERIOR_BUFFER = -10  # negative = inward
LANDSCAPE_BUFFER = 100  # outward

# Configuration for GEE assets
GEE_ASSET_PATH = 'projects/eeps-geospatial/assets/lake_geometries'

print("Configuration:")
print(f"  Years: {YEARS}")
print(f"  Chunks: {n_chunks}")
print(f"  Output: gs://{BUCKET}/{OUTPUT_PATH}")

Configuration:
  Years: [2019, 2020, 2021, 2022, 2023]
  Chunks: 21
  Output: gs://wustl-eeps-geospatial/thermokarst_lakes/exports


---
## Helper Functions

In [62]:
def load_chunk_from_bucket(chunk_id):
    """
    Load pre-computed interior and landscape geometries from GEE Assets.
    No buffer computation needed - already done.
    """
    
    # Load pre-computed geometry assets (fast!)
    interior_fc = ee.FeatureCollection(f'{GEE_ASSET_PATH}/chunk_{chunk_id:02d}_interior')
    landscape_fc = ee.FeatureCollection(f'{GEE_ASSET_PATH}/chunk_{chunk_id:02d}_landscape')
    
    # Load GeoDataFrame for metadata
    chunk_file = f'{CHUNKS_PATH}/chunk_{chunk_id:02d}.geojson'
    chunk_gdf = gpd.read_file(chunk_file)
    
    # Get bounds in WGS84 for GEE (avoids GEE edge limit!)
    chunk_gdf_wgs84 = chunk_gdf.to_crs('EPSG:4326')
    bounds_array = chunk_gdf_wgs84.total_bounds  # [minx, miny, maxx, maxy] in lon/lat
    bounds = ee.Geometry.Rectangle([
        bounds_array[0], bounds_array[1], 
        bounds_array[2], bounds_array[3]
    ])
    
    return interior_fc, landscape_fc, bounds, chunk_gdf

In [63]:
def add_lake_geometry_metrics(lakes_fc, region_bounds):
    """
    Add lake interior and landscape ring geometries
    Uses efficient spatial filtering - only masks lakes within 100m of each target lake
    """
    # Load ALPOD from GEE Asset
    all_alpod = ee.FeatureCollection('projects/eeps-geospatial/assets/ALPOD_full')
    
    def add_geometries(lake):
        lake_geom = lake.geometry()
        id = lake.get('id')
        
        # Lake interior: 10m inward buffer
        lake_interior = lake_geom.buffer(-10)
        
        # Landscape ring: 100m outward buffer
        ring_outer = lake_geom.buffer(100)
        
        # Only find lakes that intersect THIS lake's 100m buffer
        nearby_lakes = all_alpod.filterBounds(ring_outer)
        nearby_dissolved = nearby_lakes.geometry().dissolve(maxError=10)
        
        # Subtract only the nearby lakes
        landscape_ring = ring_outer.difference(nearby_dissolved)
        
        return lake.set({
            'id': id,
            'lake_interior': lake_interior,
            'landscape_ring': landscape_ring
        })
    
    return lakes_fc.map(add_geometries)

---
## Sentinel-1 Processing

In [64]:
def process_sentinel1(interior_fc, landscape_fc, year, region_bounds):
    """
    Extract S1 features using pre-computed interior and landscape FCs.
    """
    
    s1 = (ee.ImageCollection('COPERNICUS/S1_GRD')
          .filterDate(f'{year}-01-01', f'{year}-12-31')
          .filterBounds(region_bounds)
          .filter(ee.Filter.eq('instrumentMode', 'IW'))
          .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
          .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
          .filter(ee.Filter.eq('resolution_meters', 10)))
    
    def apply_angle_mask(img):
        angle = img.select('angle')
        angle_mask = angle.gt(25).And(angle.lt(50))
        return img.select(['VV', 'VH']).updateMask(angle_mask).copyProperties(img, img.propertyNames())
    
    s1 = s1.map(apply_angle_mask)
    
    def extract_s1_features(img):
        vv_img = img.select('VV')
        vh_img = img.select('VH')
        vv_vh_img = vv_img.subtract(vh_img).rename('VV_VH')
        r_band = vv_img.unitScale(-20, -5).multiply(255).rename('R')
        g_band = vh_img.unitScale(-28, -12).multiply(255).rename('G')
        b_band = vv_vh_img.unitScale(8, 18).multiply(255).rename('B')
        all_bands = vv_img.addBands(vh_img).addBands(vv_vh_img).addBands(r_band).addBands(g_band).addBands(b_band)
        
        date = img.date()
        orbit = img.get('orbitProperties_pass')
        
        # Sample both geometries
        lake_samples = all_bands.reduceRegions(
            collection=interior_fc,
            reducer=ee.Reducer.mean(),
            scale=10
        )
        
        land_samples = all_bands.reduceRegions(
            collection=landscape_fc,
            reducer=ee.Reducer.mean(),
            scale=10
        )
        
        # Join lake and landscape samples by lake id
        join_filter = ee.Filter.equals(leftField='id', rightField='id')
        joined = ee.Join.inner('lake', 'land').apply(lake_samples, land_samples, join_filter)
        
        def create_s1_feature(joined_feat):
            lake_feat = ee.Feature(joined_feat.get('lake'))
            land_feat = ee.Feature(joined_feat.get('land'))
            
            return ee.Feature(None, {
                'id': lake_feat.get('id'),
                's1_date': date.format('YYYY-MM-dd'),
                's1_doy': date.getRelative('day', 'year'),
                's1_orbit': orbit,
                'lake_vv_db': lake_feat.get('VV'),
                'lake_vh_db': lake_feat.get('VH'),
                'lake_vv_vh_db': lake_feat.get('VV_VH'),
                'lake_r': lake_feat.get('R'),
                'lake_g': lake_feat.get('G'),
                'lake_b': lake_feat.get('B'),
                'land_vv_db': land_feat.get('VV'),
                'land_vh_db': land_feat.get('VH'),
                'land_vv_vh_db': land_feat.get('VV_VH'),
                'land_r': land_feat.get('R'),
                'land_g': land_feat.get('G'),
                'land_b': land_feat.get('B')
            })
        
        return joined.map(create_s1_feature)
    
    s1_features = s1.map(extract_s1_features).flatten()
    s1_features = s1_features.filter(ee.Filter.notNull(['lake_vv_db']))
    
    return s1_features

---
## Sentinel-2 Processing

In [65]:
def compute_ndsi(img):
    """
    Compute Normalized Difference Snow Index (NDSI)
    NDSI = (Green - SWIR1) / (Green + SWIR1)
    """
    green = img.select('B3')
    swir1 = img.select('B11')
    
    ndsi = green.subtract(swir1).divide(green.add(swir1)).rename('ndsi')
    
    return img.addBands(ndsi)

def mask_s2_clouds(img):
    """
    Mask clouds using QA60 band (basic cloud mask)
    """
    qa = img.select('QA60')
    
    # Bits 10 and 11 are clouds and cirrus
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    
    # Both should be zero (clear conditions)
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(
           qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    
    return img.updateMask(mask)

def add_s2cloudless_mask(img):
    s2_cloudless = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
    cloud_prob_collection = s2_cloudless.filter(
        ee.Filter.eq('system:index', img.get('system:index'))
    )
    
    has_cloud_data = cloud_prob_collection.size().gt(0)
    
    def apply_s2cloudless_mask():
        cloud_prob = cloud_prob_collection.first().select('probability')
        is_clear = cloud_prob.lt(S2_CLOUD_PROB_THRESHOLD)
        return img.updateMask(is_clear)
    
    def use_qa60_only():
        return img  # Already has QA60 mask
    
    return ee.Image(ee.Algorithms.If(
        has_cloud_data,
        apply_s2cloudless_mask(),
        use_qa60_only()
    ))

In [66]:
def process_sentinel2(interior_fc, year, region_bounds):
    """
    Extract S2 features using pre-computed interior FC.
    """
    
    s2 = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
          .filterDate(f'{year}-01-01', f'{year}-12-31')
          .filterBounds(region_bounds)
          .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', S2_CLOUD_THRESHOLD)))
    
    s2_cloudless = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
                    .filterDate(f'{year}-01-01', f'{year}-12-31')
                    .filterBounds(region_bounds))
    
    def add_cloud_and_ndsi_bands(img):
        img_date = img.date()
        cloud_prob_img = s2_cloudless.filterDate(
            img_date, img_date.advance(1, 'day')
        ).first()
        
        cloud_prob = ee.Image(ee.Algorithms.If(
            cloud_prob_img,
            ee.Image(cloud_prob_img).select('probability'),
            ee.Image.constant(0).rename('probability')
        ))
        
        qa = img.select('QA60')
        cloud_bit_mask = 1 << 10
        cirrus_bit_mask = 1 << 11
        qa_clear = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(
                   qa.bitwiseAnd(cirrus_bit_mask).eq(0))
        
        s2cloudless_clear = cloud_prob.lt(S2_CLOUD_PROB_THRESHOLD)
        combined_clear = qa_clear.And(s2cloudless_clear)
        
        ndsi = img.normalizedDifference(['B3', 'B11']).rename('ndsi')
        ndsi_masked = ndsi.updateMask(combined_clear)
        ice_binary = ndsi_masked.gt(S2_NDSI_THRESHOLD).rename('ice_binary')
        cloud_mask = combined_clear.Not().rename('is_cloud')
        
        return img.addBands(ndsi_masked).addBands(ice_binary).addBands(cloud_mask)
    
    s2 = s2.map(add_cloud_and_ndsi_bands)
    
    def extract_s2_features(img):
        date = img.date()
        
        # Stack bands for single reduceRegions call
        bands = img.select(['ndsi', 'ice_binary', 'is_cloud'])
        
        # Combined reducer: mean + count + stdDev
        reducer = ee.Reducer.mean().combine(
            ee.Reducer.count(), '', True
        ).combine(
            ee.Reducer.stdDev(), '', True
        )
        
        samples = bands.reduceRegions(
            collection=interior_fc,
            reducer=reducer,
            scale=20
        )
        
        def create_s2_feature(feat):
            # Get all metrics with safe null handling
            ndsi_mean = feat.get('ndsi')
            ndsi_count = feat.get('ndsi_count')
            ndsi_std = feat.get('ndsi_stdDev')
            ice_fraction = feat.get('ice_binary')
            cloud_fraction = feat.get('is_cloud')
            
            # Safe conversions with null checks
            cloud_pct = ee.Algorithms.If(
                cloud_fraction,
                ee.Number(cloud_fraction).multiply(100),
                None
            )
            
            ice_fraction_safe = ee.Algorithms.If(
                ndsi_mean,
                ice_fraction,
                None
            )
            
            pixel_count_safe = ee.Algorithms.If(
                ndsi_count,
                ee.Number(ndsi_count).toInt(),
                None
            )
            
            ndsi_std_safe = ee.Algorithms.If(
                ndsi_std,
                ndsi_std,
                None
            )
            
            return ee.Feature(None, {
                'id': feat.get('id'),
                's2_date': date.format('YYYY-MM-dd'),
                's2_doy': date.getRelative('day', 'year'),
                's2_ndsi_mean': ndsi_mean,
                's2_ndsi_std': ndsi_std_safe,
                's2_ice_fraction': ice_fraction_safe,
                's2_pixel_count': pixel_count_safe,
                's2_cloud_pct': cloud_pct
            })
        
        return samples.map(create_s2_feature)
    
    s2_features = s2.map(extract_s2_features).flatten()
    s2_features = s2_features.filter(ee.Filter.notNull(['s2_ndsi_mean']))
    
    return s2_features

---
## ERA5 Temperature Processing

In [67]:
def process_era5_temperature(lakes_fc, year):
    """
    Process ERA5-Land temperature data - OPTIMIZED VERSION
    Pre-computes daily means once, then samples all lakes in batch
    Much faster than computing daily mean separately for each lake
    """
    print(f"  Loading ERA5 hourly data for {year}...")
    
    # Load ERA5-Land hourly data
    era5_hourly = (ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')
                   .filterDate(f'{year}-01-01', f'{year}-12-31')
                   .select('temperature_2m'))
    
    # Convert to Celsius
    def to_celsius(img):
        temp_c = img.subtract(273.15).rename('temp_c')
        return temp_c.copyProperties(img, ['system:time_start'])
    
    era5_hourly = era5_hourly.map(to_celsius)
    
    print(f"  Pre-computing daily means...")
    
    # Determine number of days in year (handle leap years)
    is_leap = ee.Number(year).mod(4).eq(0).And(
        ee.Number(year).mod(100).neq(0).Or(
            ee.Number(year).mod(400).eq(0)
        )
    )
    n_days = ee.Number(ee.Algorithms.If(is_leap, 366, 365))
    
    # Pre-compute daily means for entire year (365 or 366 images)
    def compute_daily_mean(day):
        day = ee.Number(day)
        date = ee.Date.fromYMD(year, 1, 1).advance(day.subtract(1), 'day')
        next_date = date.advance(1, 'day')
        
        # Get all hourly images for this day
        daily_collection = era5_hourly.filterDate(date, next_date)
        
        # Check if we have data
        has_data = daily_collection.size().gt(0)
        
        # Compute mean if data exists, otherwise use missing flag
        daily_mean = ee.Image(ee.Algorithms.If(
            has_data,
            daily_collection.mean(),
            ee.Image.constant(-9999).rename('temp_c')
        ))
        
        return daily_mean.set({
            'system:time_start': date.millis(),
            'doy': day,
            'date': date.format('YYYY-MM-dd')
        })
    
    days = ee.List.sequence(1, n_days)
    era5_daily = ee.ImageCollection.fromImages(days.map(compute_daily_mean))
    
    print(f"  Sampling all lakes from daily means...")
    
    # Now sample ALL lakes from each daily mean (batch operation)
    def sample_all_lakes(daily_img):
        doy = daily_img.get('doy')
        date = daily_img.get('date')
        
        # Sample ALL lakes at once using reduceRegions
        samples = daily_img.reduceRegions(
            collection=lakes_fc,
            reducer=ee.Reducer.first(),  # Get pixel value at centroid
            scale=11000  # ERA5-Land resolution
        )
        
        # Add date info to each sampled feature
        def add_date_info(feat):
            # Get the temperature value (from 'first' property created by reducer)
            # Use ee.Algorithms.If to provide default if missing
            temp_value = ee.Algorithms.If(
                feat.propertyNames().contains('first'),
                feat.get('first'),
                -9999
            )
            
            return feat.set({
                'era5_date': date,
                'era5_doy': doy,
                'temp_c': temp_value
            })
        
        return samples.map(add_date_info)
    
    # Process all daily images
    era5_features = era5_daily.map(sample_all_lakes).flatten()
    
    return era5_features

---
## Show chunk statistics

In [68]:
# Load chunk statistics to know how many chunks we have
chunk_stats = pd.read_csv(f'gs://{BUCKET}/{BASE_PATH}/processed/chunk_statistics.csv')
n_chunks = len(chunk_stats)

print(f"Total chunks to process: {n_chunks}")
print(f"Years to process: {YEARS}")
print(f"Total exports: {n_chunks * len(YEARS) * 3} (chunks × years × 3 datasets)")
print("\nChunk statistics:")
print(chunk_stats[['chunk_id', 'n_lakes', 'lat_min', 'lat_max', 'lon_min', 'lon_max']])

Total chunks to process: 21
Years to process: [2019, 2020, 2021, 2022, 2023]
Total exports: 315 (chunks × years × 3 datasets)

Chunk statistics:
    chunk_id  n_lakes    lat_min    lat_max     lon_min     lon_max
0          0     1659  69.003753  70.805772 -157.460685 -156.323643
1          1     1402  69.003327  70.498668 -150.588202 -149.501159
2          2     2172  69.000561  70.506201 -149.575529 -148.460811
3          3     1002  69.002549  70.662131 -161.092265 -159.724530
4          4     2243  70.139174  71.157997 -155.626734 -154.362639
5          5      408  69.061209  70.123854 -144.857630 -141.021667
6          6     1726  69.886316  70.909059 -153.572246 -152.259761
7          7     1704  69.002644  70.811790 -158.909885 -157.856992
8          8      753  69.019065  70.186629 -147.158786 -144.887360
9          9      460  69.027426  70.075681 -163.578774 -162.004458
10        10     1440  69.013772  70.328129 -153.976923 -152.575637
11        11     1392  69.000974  70.21

---
## ERA5 Temperature export (via xarray)

In [69]:
import geopandas as gpd

chunk_gdf = gpd.read_file('gs://wustl-eeps-geospatial/thermokarst_lakes/processed/chunks/chunk_00.geojson')

print("Columns after reading back:")
print(chunk_gdf.columns.tolist())
print("\nFirst row:")
print(chunk_gdf.iloc[0])

Columns after reading back:
['id', 'chunk_id', 'lake_area_m2', 'lake_area_km2', 'lake_perim_m', 'circularity', 'shoreline_dev', 'convexity', 'centroid_lon', 'centroid_lat', 'geometry']

First row:
id                                                          278001
chunk_id                                                         0
lake_area_m2                                          28192.123629
lake_area_km2                                             0.028192
lake_perim_m                                            829.785235
circularity                                               0.514525
shoreline_dev                                             1.394109
convexity                                                 0.910838
centroid_lon                                           -157.027101
centroid_lat                                             70.805772
geometry         POLYGON ((-113496.97826215002 2315713.90602798...
Name: 0, dtype: object


In [70]:
# ============================================================
# ERA5 Export Control
# ============================================================
RUN_ERA5_EXPORT = False  # Set to True to run ERA5 export

if not RUN_ERA5_EXPORT:
    print("Skipping ERA5 export (RUN_ERA5_EXPORT = False)")
    print("ERA5 data already exported - see gs://wustl-eeps-geospatial/thermokarst_lakes/exports/YYYY/chunk_XX/era5_data.csv")

Skipping ERA5 export (RUN_ERA5_EXPORT = False)
ERA5 data already exported - see gs://wustl-eeps-geospatial/thermokarst_lakes/exports/YYYY/chunk_XX/era5_data.csv


In [71]:
# ============================================================
# ERA5 Temperature Export (via xarray) 
# ============================================================
if RUN_ERA5_EXPORT:
    print("="*60)
    print("DOWNLOADING ERA5 TEMPERATURE DATA")
    print("="*60)

    era5_start = time.time()

    # Load chunk info
    print("Step 1: Loading lake locations by chunk...")
    chunks_info = []

    for chunk_id in range(n_chunks):
        chunk_file = f'{CHUNKS_PATH}/chunk_{chunk_id:02d}.geojson'
        chunk_gdf = gpd.read_file(chunk_file)
       
        
        # Keep original -180:180 coordinates
        chunks_info.append({
            'chunk_id': chunk_id,
            'gdf': chunk_gdf[['id', 'centroid_lon', 'centroid_lat']],
            'n_lakes': len(chunk_gdf)
        })

    total_lakes = sum(c['n_lakes'] for c in chunks_info)
    print(f"  Loaded {total_lakes:,} lakes across {len(chunks_info)} chunks\n")

    # Download ERA5
    print("Step 2: Accessing ERA5 dataset...")
    ERA5_URL = 'gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3'
    ds = xr.open_zarr(ERA5_URL, chunks={'time': 100}, consolidated=True)

    # Check coordinate system
    print("  Checking coordinates...")
    lat_min, lat_max = float(ds.latitude.min()), float(ds.latitude.max())
    lon_min, lon_max = float(ds.longitude.min()), float(ds.longitude.max())
    print(f"  Latitude: {lat_min:.1f} to {lat_max:.1f}")
    print(f"  Longitude: {lon_min:.1f} to {lon_max:.1f}\n")

    # Convert Alaska longitudes if needed
    alaska_lon_min, alaska_lon_max = -164, -140
    if lon_min >= 0:  # Dataset uses 0-360
        alaska_lon_min = alaska_lon_min % 360  # 196
        alaska_lon_max = alaska_lon_max % 360  # 220
        print(f"  Converting to 0-360: Alaska is {alaska_lon_min}-{alaska_lon_max}E\n")

    print("Step 3: Selecting North Slope region...")
    # Select with proper ordering
    if ds.latitude[0] > ds.latitude[-1]:  # Descending
        lat_slice = slice(72, 69)
    else:  # Ascending
        lat_slice = slice(69, 72)

    ds_subset = ds['2m_temperature'].sel(
        latitude=lat_slice,
        longitude=slice(alaska_lon_min, alaska_lon_max),
        time=slice(f'{YEARS[0]}-01-01', f'{YEARS[-1]}-12-31')
    )

    print(f"  Selected: {len(ds_subset.latitude)} lats x {len(ds_subset.longitude)} lons x {len(ds_subset.time)} hours\n")

    if len(ds_subset.latitude) == 0 or len(ds_subset.longitude) == 0:
        raise ValueError("Selection returned empty dataset - check coordinate ranges")

    print("Step 4: Computing daily means...")
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        # Compute and load into memory (avoids dask chunking issues)
        ds_daily = ds_subset.resample(time='1D').mean().compute()
        ds_daily_celsius = ds_daily - 273.15

    print(f"  {len(ds_daily_celsius.time)} daily means computed\n")

    print("Step 5: Processing each chunk...")

    for chunk_info in chunks_info:
        chunk_id = chunk_info['chunk_id']
        chunk_gdf = chunk_info['gdf']
        n_lakes = chunk_info['n_lakes']
        
        print(f"  Chunk {chunk_id:02d} ({n_lakes} lakes)...", end=' ')
        
        # Convert lake coordinates if needed
        lake_lons = chunk_gdf['centroid_lon'].values
        if alaska_lon_min > 180:  # Dataset uses 0-360
            lake_lons = lake_lons % 360
        
        lake_lats = xr.DataArray(chunk_gdf['centroid_lat'].values, dims=['lake'])
        lake_lons_da = xr.DataArray(lake_lons, dims=['lake'])
        
        # Interpolate
        lake_temps = ds_daily_celsius.interp(
            latitude=lake_lats,
            longitude=lake_lons_da,
            method='linear'
        )
        
        # Convert to DataFrame
        df_chunk = lake_temps.to_dataframe(name='temp_c').reset_index()
        df_chunk['id'] = chunk_gdf['id'].values[df_chunk['lake']]
        df_chunk['era5_date'] = df_chunk['time'].dt.strftime('%Y-%m-%d')
        df_chunk['era5_doy'] = df_chunk['time'].dt.dayofyear
        df_chunk['year'] = df_chunk['time'].dt.year
        df_chunk = df_chunk[['id', 'era5_date', 'era5_doy', 'temp_c', 'year']]
        
        # Export by year
        for year in YEARS:
            df_year = df_chunk[df_chunk['year'] == year].copy()
            df_year = df_year.drop(columns=['year']).sort_values(['id', 'era5_doy'])
            output_file = f'gs://{BUCKET}/{OUTPUT_PATH}/{year}/chunk_{chunk_id:02d}/era5_data.csv'
            df_year.to_csv(output_file, index=False)
        
        print("Done")

    era5_time = time.time() - era5_start

    print(f"\n{'='*60}")
    print("ERA5 COMPLETE!")
    print(f"{'='*60}")
    print(f"Time: {era5_time/60:.1f} minutes")
    print(f"Files: {len(chunks_info) * len(YEARS)} CSV files")
else:
    print("Skipping ERA5 export (RUN_ERA5_EXPORT = False)")

Skipping ERA5 export (RUN_ERA5_EXPORT = False)


In [72]:
import geopandas as gpd

CHUNKS_PATH = 'gs://wustl-eeps-geospatial/thermokarst_lakes/processed/chunks'

# Check chunk_00
chunk_gdf = gpd.read_file(f'{CHUNKS_PATH}/chunk_00.geojson')

print("Columns in chunk_00.geojson:")
print(chunk_gdf.columns.tolist())
print("\nFirst few rows:")
print(chunk_gdf.head())

Columns in chunk_00.geojson:
['id', 'chunk_id', 'lake_area_m2', 'lake_area_km2', 'lake_perim_m', 'circularity', 'shoreline_dev', 'convexity', 'centroid_lon', 'centroid_lat', 'geometry']

First few rows:
       id  chunk_id  lake_area_m2  lake_area_km2  lake_perim_m  circularity  \
0  278001         0  2.819212e+04       0.028192    829.785235     0.514525   
1  279125         0  7.143037e+04       0.071430   1458.696235     0.421855   
2  280053         0  7.732235e+06       7.732235  16148.322690     0.372615   
3  280110         0  4.257919e+06       4.257919  12113.186764     0.364662   
4  280158         0  4.343958e+04       0.043440   1009.096047     0.536081   

   shoreline_dev  convexity  centroid_lon  centroid_lat  \
0       1.394109   0.910838   -157.027101     70.805772   
1       1.539638   0.890586   -157.035125     70.766392   
2       1.638211   0.961461   -156.978301     70.761569   
3       1.655979   0.886562   -157.020026     70.753066   
4       1.365793   0.933048

---
## S1/S2 exports (via GEE)

**WARNING:** This will prep ~210 export tasks (21 chunks × 5 years × 2 datasets each)

In [73]:
# ============================================================
# Prep S1/S2 exports
# ERA5 already exported above
# ============================================================

all_exports = []
total_start = time.time()

for chunk_id in range(n_chunks):
    print(f"\n{'='*60}")
    print(f"Processing Chunk {chunk_id}")
    print(f"{'='*60}")
    
    chunk_start = time.time()
    
    # Load chunk with pre-computed geometries
    interior_fc, landscape_fc, chunk_bounds, chunk_gdf = load_chunk_from_bucket(chunk_id)
    
    print(f"  {len(chunk_gdf)} lakes (geometries pre-computed)")
    
    for year in YEARS:
        print(f"  Preparing {year}...", end=' ')
        
        # Process S1 and S2 with pre-computed FCs
        s1_features = process_sentinel1(interior_fc, landscape_fc, year, chunk_bounds)
        s2_features = process_sentinel2(interior_fc, year, chunk_bounds)
        
        # Create export tasks
        s1_task = ee.batch.Export.table.toCloudStorage(
            collection=s1_features,
            description=f'S1_chunk{chunk_id:02d}_{year}',
            bucket=BUCKET,
            fileNamePrefix=f'{OUTPUT_PATH}/{year}/chunk_{chunk_id:02d}/s1_data',
            fileFormat='CSV'
        )
        
        s2_task = ee.batch.Export.table.toCloudStorage(
            collection=s2_features,
            description=f'S2_chunk{chunk_id:02d}_{year}',
            bucket=BUCKET,
            fileNamePrefix=f'{OUTPUT_PATH}/{year}/chunk_{chunk_id:02d}/s2_data',
            fileFormat='CSV'
        )
        
        all_exports.append({
            'chunk_id': chunk_id,
            'year': year,
            'type': 'S1',
            'task': s1_task
        })
        all_exports.append({
            'chunk_id': chunk_id,
            'year': year,
            'type': 'S2',
            'task': s2_task
        })
        
        print("Done")
    
    chunk_time = time.time() - chunk_start
    print(f"  Chunk {chunk_id} complete ({chunk_time:.1f}s)")

total_time = time.time() - total_start
print(f"\n{'='*60}")
print(f"ALL EXPORTS PREPARED: {len(all_exports)} tasks")
print(f"{'='*60}")
print(f"Total preparation time: {total_time/60:.1f} minutes")
print(f"\nReady to start {len(all_exports)} S1/S2 exports.")
print(f"ERA5 already complete (63 files).")


Processing Chunk 0
  1659 lakes (geometries pre-computed)
  Preparing 2019... Done
  Preparing 2020... Done
  Preparing 2021... Done
  Preparing 2022... Done
  Preparing 2023... Done
  Chunk 0 complete (0.8s)

Processing Chunk 1
  1402 lakes (geometries pre-computed)
  Preparing 2019... Done
  Preparing 2020... Done
  Preparing 2021... Done
  Preparing 2022... Done
  Preparing 2023... Done
  Chunk 1 complete (1.3s)

Processing Chunk 2
  2172 lakes (geometries pre-computed)
  Preparing 2019... Done
  Preparing 2020... Done
  Preparing 2021... Done
  Preparing 2022... Done
  Preparing 2023... Done
  Chunk 2 complete (3.5s)

Processing Chunk 3
  1002 lakes (geometries pre-computed)
  Preparing 2019... Done
  Preparing 2020... Done
  Preparing 2021... Done
  Preparing 2022... Done
  Preparing 2023... Done
  Chunk 3 complete (1.0s)

Processing Chunk 4
  2243 lakes (geometries pre-computed)
  Preparing 2019... Done
  Preparing 2020... Done
  Preparing 2021... Done
  Preparing 2022... Done
 

In [ ]:
# Start all exports
print(f"Starting {len(all_exports)} tasks...")
started = 0

for i, export in enumerate(all_exports):
    try:
        export['task'].start()
        started += 1
        
        if started % 10 == 0:
            print(f"  Started {started} tasks...")
            time.sleep(1)
    except Exception as e:
        print(f"Error starting task {i} ({export['type']} chunk {export['chunk_id']} {export['year']}): {e}")

print(f"\nStarted {started}/{len(all_exports)} tasks")
print("Monitor at: https://code.earthengine.google.com/tasks")

Starting 210 tasks...
  Started 10 tasks...
  Started 20 tasks...
  Started 30 tasks...
  Started 40 tasks...
  Started 50 tasks...
  Started 60 tasks...


---
## Monitor GEE Export Progress

In [ ]:
# Check status of GEE exports
def check_export_status():
    status_summary = {
        'READY': 0,
        'RUNNING': 0,
        'COMPLETED': 0,
        'FAILED': 0,
        'CANCELLED': 0
    }
    
    for exp in all_exports:
        status = exp['task'].status()['state']
        status_summary[status] = status_summary.get(status, 0) + 1
    
    print(f"Export Status Summary:")
    print(f"  Total tasks: {len(all_exports)}")
    for state, count in status_summary.items():
        if count > 0:
            print(f"    {state}: {count}")
    
    return status_summary

# Run this cell periodically to check progress
check_export_status()

---
## Summary

This notebook exports:

**Sentinel-1**: 
- Lake interior: VV, VH backscatter (dB), VV-VH ratio, RGB features (normalized 0-255)
- Landscape ring: VV, VH backscatter (dB), VV-VH ratio, RGB features (normalized 0-255)
- Both ASCENDING and DESCENDING orbits
- Angle mask applied (25-50 degrees)

**Sentinel-2**: 
- s2_ndsi_mean: Mean NDSI value across lake pixels (continuous, -1 to 1)
- s2_ndsi_std: Standard deviation of NDSI across lake pixels (measure of ice/water mixing)
- s2_ice_fraction: Fraction of lake pixels with NDSI > 0.4 (continuous, 0 to 1)
- s2_pixel_count: Number of valid (non-cloud) pixels used in calculation
- s2_cloud_pct: Percentage of lake masked by clouds (0-100, for QC filtering)
- Dual cloud masking: QA60 + s2cloudless

**ERA5**: 
- Daily mean 2m air temperature at lake centroids (Celsius)

**For:**
- 21 spatial chunks
- 5 years (2019, 2020, 2021, 2022, 2023)
- ~31,000 North Slope lakes (>0.02 km²)

**Output structure:**
```
gs://wustl-eeps-geospatial/thermokarst_lakes/exports/
├── 2019/
│   ├── chunk_00/
│   │   ├── s1_data.csv
│   │   ├── s2_data.csv
│   │   └── era5_data.csv
│   ├── chunk_01/
│   └── ...
├── 2020/
├── 2021/
├── 2022/
└── 2023/
```

**Next step:** Combine CSVs and run ice detection algorithm (Notebook 03)